In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests

In [3]:
# Retrieve NASA News page
response = requests.get('https://mars.nasa.gov/news/')

In [4]:
#BeautifulSoup html parser
soup = BeautifulSoup(response.text, 'html.parser')

In [52]:
results = soup.find_all('div', class_= 'content_title')[0].text
results

"\n\nNASA's Perseverance Drives on Mars' Terrain for First Time\n\n"

In [58]:
title = results.strip()

title


"NASA's Perseverance Drives on Mars' Terrain for First Time"

In [64]:
results = soup.find_all('div', class_='rollover_description_inner')[0].text

In [66]:
tag = results.strip()

tag

'The first trek of the agency’s largest, most advanced rover yet on the Red Planet marks a major milestone before science operations get under way.'